In [ ]:
pip install transformers datasets evaluate rouge_score pandas tqdm accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler
)
from torch.optim import AdamW
import evaluate
from tqdm import tqdm
import pandas as pd
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

TEACHER_MODEL = "openai-community/gpt2-xl"
STUDENT_MODEL = "openai-community/gpt2-medium"

MAX_LENGTH = 256
BATCH_SIZE = 1
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3
TEMPERATURE = 2.0
ALPHA = 0.5


def load_adversarialqa_sample(sample_size=100):
    from datasets import load_dataset
    dataset = load_dataset("adversarial_qa", "adversarialQA")
    return dataset["train"].select(range(sample_size))


class KnowledgeDistillationDataset(Dataset):
    def __init__(self, dataset, teacher_tokenizer, student_tokenizer, max_length):
        self.dataset = dataset
        self.teacher_tokenizer = teacher_tokenizer
        self.student_tokenizer = student_tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        question = item["question"]
        context = item["context"]
        answer = item["answers"]["text"][0] if item["answers"]["text"] else ""

        prompt = f"Answer the question based on the context.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"

        teacher_inputs = self.teacher_tokenizer(prompt, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        student_inputs = self.student_tokenizer(prompt, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        teacher_labels = self.teacher_tokenizer(answer, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        student_labels = self.student_tokenizer(answer, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "teacher_input_ids": teacher_inputs.input_ids.squeeze(),
            "teacher_attention_mask": teacher_inputs.attention_mask.squeeze(),
            "teacher_labels": teacher_labels.input_ids.squeeze(),
            "student_input_ids": student_inputs.input_ids.squeeze(),
            "student_attention_mask": student_inputs.attention_mask.squeeze(),
            "student_labels": student_labels.input_ids.squeeze(),
            "prompt": prompt,
            "answer": answer
        }


def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha, ignore_index):
    vocab_size = student_logits.size(-1)
    if teacher_logits.size(-1) > vocab_size:
        teacher_logits = teacher_logits[..., :vocab_size]

    soft_targets = F.softmax(teacher_logits / temperature, dim=-1)
    soft_prob = F.log_softmax(student_logits / temperature, dim=-1)
    kd_loss = F.kl_div(soft_prob, soft_targets, reduction='batchmean') * (temperature ** 2)
    ce_loss = F.cross_entropy(student_logits.view(-1, vocab_size), labels.view(-1), ignore_index=ignore_index)
    return alpha * ce_loss + (1 - alpha) * kd_loss, ce_loss, kd_loss


def train_model(teacher_model, student_model, train_loader, optimizer, scheduler, ignore_index):
    student_model.train()
    teacher_model.eval()
    total_loss = total_ce_loss = total_kd_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")

    for batch in progress_bar:
        student_input_ids = batch["student_input_ids"].to(device)
        student_attention_mask = batch["student_attention_mask"].to(device)
        student_labels = batch["student_labels"].to(device)
        teacher_input_ids = batch["teacher_input_ids"].to(device)
        teacher_attention_mask = batch["teacher_attention_mask"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(teacher_input_ids, attention_mask=teacher_attention_mask).logits

        student_outputs = student_model(student_input_ids, attention_mask=student_attention_mask)
        student_logits = student_outputs.logits

        loss, ce_loss, kd_loss = distillation_loss(student_logits, teacher_logits, student_labels, TEMPERATURE, ALPHA, ignore_index)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_ce_loss += ce_loss.item()
        total_kd_loss += kd_loss.item()
        progress_bar.set_postfix({
            "Loss": total_loss / (progress_bar.n + 1),
            "CE": total_ce_loss / (progress_bar.n + 1),
            "KD": total_kd_loss / (progress_bar.n + 1)
        })


def evaluate_model(model, tokenizer, eval_dataset, name="Model"):
    from collections import Counter
    import string, re
    from unicodedata import normalize

    def normalize_text(text):
        text = text.lower()
        text = normalize("NFKD", text)
        text = re.sub(r"\b(a|an|the)\b", " ", text)
        text = "".join(ch for ch in text if ch not in string.punctuation)
        text = " ".join(text.split())
        return text

    def compute_f1(pred, truth):
        pred_tokens = normalize_text(pred).split()
        truth_tokens = normalize_text(truth).split()
        common = Counter(pred_tokens) & Counter(truth_tokens)
        num_same = sum(common.values())
        if num_same == 0:
            return 0.0
        precision = num_same / len(pred_tokens)
        recall = num_same / len(truth_tokens)
        return 2 * precision * recall / (precision + recall)

    def exact_match(pred, truth):
        return normalize_text(pred) == normalize_text(truth)

    model.eval()
    rouge_metric = evaluate.load("rouge")
    bleu_metric = evaluate.load("bleu")

    results = []
    loader = DataLoader(eval_dataset, batch_size=1)
    all_preds, all_refs = [], []
    em_total, f1_total = 0, 0

    for idx, batch in enumerate(tqdm(loader, desc=f"Evaluating {name}")):
        input_ids = batch[f"{name.lower()}_input_ids"].to(device)
        attention_mask = batch[f"{name.lower()}_attention_mask"].to(device)
        prompt = batch["prompt"]
        true_answer = batch["answer"]

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=64)
            prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        generated_answer = prediction.replace(prompt[0], "").strip()
        reference = true_answer[0].strip()

        em = int(exact_match(generated_answer, reference))
        f1 = compute_f1(generated_answer, reference)
        em_total += em
        f1_total += f1

        results.append({
            "prompt": prompt[0],
            "reference": reference,
            "prediction": generated_answer,
            "EM": em,
            "F1": f1
        })

        all_preds.append(generated_answer)
        all_refs.append(reference)

    rouge_scores = rouge_metric.compute(predictions=all_preds, references=all_refs)
    bleu_scores = bleu_metric.compute(predictions=all_preds, references=[[ref] for ref in all_refs])

    avg_em = em_total / len(eval_dataset)
    avg_f1 = f1_total / len(eval_dataset)

    return results, {
        "rouge": rouge_scores,
        "bleu": bleu_scores,
        "em": avg_em,
        "f1": avg_f1
    }


def print_metrics(name, scores):
    print(f"\n{name} Evaluation:")
    print(f"  Exact Match (EM): {scores['em']:.2f}")
    print(f"  F1 Score: {scores['f1']:.2f}")
    print(f"  ROUGE:")
    for k, v in scores["rouge"].items():
        print(f"    {k}: {v:.2f}")
    print(f"  BLEU: {scores['bleu']['bleu']:.2f}")


def main():
    print(" Loading dataset...")
    dataset = load_adversarialqa_sample(100)

    print(" Loading teacher model...")
    teacher_tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL)
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token
    teacher_model = AutoModelForCausalLM.from_pretrained(TEACHER_MODEL).to(device)

    print(" Loading student model...")
    student_tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)
    student_tokenizer.pad_token = student_tokenizer.eos_token
    student_pad_token_id = student_tokenizer.pad_token_id
    student_model = AutoModelForCausalLM.from_pretrained(STUDENT_MODEL).to(device)

    print(" Preparing dataset...")
    kd_dataset = KnowledgeDistillationDataset(dataset, teacher_tokenizer, student_tokenizer, MAX_LENGTH)
    train_size = int(0.8 * len(kd_dataset))
    train_dataset, eval_dataset = random_split(kd_dataset, [train_size, len(kd_dataset) - train_size])
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    optimizer = AdamW(student_model.parameters(), lr=LEARNING_RATE)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*NUM_EPOCHS)

    for epoch in range(NUM_EPOCHS):
        print(f"\n Epoch {epoch+1}/{NUM_EPOCHS}")
        train_model(teacher_model, student_model, train_loader, optimizer, scheduler, student_pad_token_id)

        out_dir = f"student_checkpoint_epoch_{epoch+1}"
        os.makedirs(out_dir, exist_ok=True)
        student_model.save_pretrained(out_dir)
        student_tokenizer.save_pretrained(out_dir)

    print("\n Evaluating models...")
    student_results, student_scores = evaluate_model(student_model, student_tokenizer, eval_dataset, name="Student")
    teacher_results, teacher_scores = evaluate_model(teacher_model, teacher_tokenizer, eval_dataset, name="Teacher")

    print_metrics("Student", student_scores)
    print_metrics("Teacher", teacher_scores)

    pd.DataFrame(student_results).to_csv("distillation_eval_results_adversarialqa.csv", index=False)
    print("\n Results saved to distillation_eval_results_adversarialqa.csv")


if __name__ == "__main__":
    main()


Using device: cuda
 Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.35M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/495k [00:00<?, ?B/s]

...


 Epoch 3/3


Training: 100%|██████████| 80/80 [00:45<00:00,  1.76it/s, Loss=48.7, CE=9.49, KD=87.8]



 Evaluating models...


Evaluating Teacher: 100%|██████████| 20/20 [01:02<00:00,  3.14s/it]



Student Evaluation:
  Exact Match (EM): 0.00
  F1 Score: 0.04
  ROUGE:
    rouge1: 0.04
    rouge2: 0.01
    rougeL: 0.04
    rougeLsum: 0.04
  BLEU: 0.01

Teacher Evaluation:
  Exact Match (EM): 0.00
  F1 Score: 0.05
  ROUGE:
    rouge1: 0.05
    rouge2: 0.02
    rougeL: 0.05
    rougeLsum: 0.05
  BLEU: 0.01

 Results saved to distillation_eval_results_adversarialqa.csv


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler
)
from torch.optim import AdamW
import evaluate
from tqdm import tqdm
import pandas as pd
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

TEACHER_MODEL = "openai-community/gpt2-xl"
STUDENT_MODEL = "openai-community/gpt2-medium"

MAX_LENGTH = 256
BATCH_SIZE = 1
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3
TEMPERATURE = 2.0
ALPHA = 0.5


def load_adversarialqa_sample(sample_size=100):
    from datasets import load_dataset
    dataset = load_dataset("adversarial_qa", "adversarialQA")
    return dataset["train"].select(range(sample_size))


class KnowledgeDistillationDataset(Dataset):
    def __init__(self, dataset, teacher_tokenizer, student_tokenizer, max_length):
        self.dataset = dataset
        self.teacher_tokenizer = teacher_tokenizer
        self.student_tokenizer = student_tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        question = item["question"]
        context = item["context"]
        answer = item["answers"]["text"][0] if item["answers"]["text"] else ""

        prompt = f"Answer the question based on the context.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"

        teacher_inputs = self.teacher_tokenizer(prompt, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        student_inputs = self.student_tokenizer(prompt, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        teacher_labels = self.teacher_tokenizer(answer, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        student_labels = self.student_tokenizer(answer, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        return {
            "teacher_input_ids": teacher_inputs.input_ids.squeeze(),
            "teacher_attention_mask": teacher_inputs.attention_mask.squeeze(),
            "teacher_labels": teacher_labels.input_ids.squeeze(),
            "student_input_ids": student_inputs.input_ids.squeeze(),
            "student_attention_mask": student_inputs.attention_mask.squeeze(),
            "student_labels": student_labels.input_ids.squeeze(),
            "prompt": prompt,
            "answer": answer
        }


def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha, ignore_index):
    vocab_size = student_logits.size(-1)
    if teacher_logits.size(-1) > vocab_size:
        teacher_logits = teacher_logits[..., :vocab_size]

    soft_targets = F.softmax(teacher_logits / temperature, dim=-1)
    soft_prob = F.log_softmax(student_logits / temperature, dim=-1)
    kd_loss = F.kl_div(soft_prob, soft_targets, reduction='batchmean') * (temperature ** 2)
    ce_loss = F.cross_entropy(student_logits.view(-1, vocab_size), labels.view(-1), ignore_index=ignore_index)
    return alpha * ce_loss + (1 - alpha) * kd_loss, ce_loss, kd_loss


def train_model(teacher_model, student_model, train_loader, optimizer, scheduler, ignore_index):
    student_model.train()
    teacher_model.eval()
    total_loss = total_ce_loss = total_kd_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")

    for batch in progress_bar:
        student_input_ids = batch["student_input_ids"].to(device)
        student_attention_mask = batch["student_attention_mask"].to(device)
        student_labels = batch["student_labels"].to(device)
        teacher_input_ids = batch["teacher_input_ids"].to(device)
        teacher_attention_mask = batch["teacher_attention_mask"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(teacher_input_ids, attention_mask=teacher_attention_mask).logits

        student_outputs = student_model(student_input_ids, attention_mask=student_attention_mask)
        student_logits = student_outputs.logits

        loss, ce_loss, kd_loss = distillation_loss(student_logits, teacher_logits, student_labels, TEMPERATURE, ALPHA, ignore_index)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_ce_loss += ce_loss.item()
        total_kd_loss += kd_loss.item()
        progress_bar.set_postfix({
            "Loss": total_loss / (progress_bar.n + 1),
            "CE": total_ce_loss / (progress_bar.n + 1),
            "KD": total_kd_loss / (progress_bar.n + 1)
        })


def evaluate_model(model, tokenizer, eval_dataset, name="Model"):
    from collections import Counter
    import string, re
    from unicodedata import normalize

    def normalize_text(text):
        text = text.lower()
        text = normalize("NFKD", text)
        text = re.sub(r"\b(a|an|the)\b", " ", text)
        text = "".join(ch for ch in text if ch not in string.punctuation)
        text = " ".join(text.split())
        return text

    def compute_f1(pred, truth):
        pred_tokens = normalize_text(pred).split()
        truth_tokens = normalize_text(truth).split()
        common = Counter(pred_tokens) & Counter(truth_tokens)
        num_same = sum(common.values())
        if num_same == 0:
            return 0.0
        precision = num_same / len(pred_tokens)
        recall = num_same / len(truth_tokens)
        return 2 * precision * recall / (precision + recall)

    def exact_match(pred, truth):
        return normalize_text(pred) == normalize_text(truth)

    model.eval()
    results = []
    loader = DataLoader(eval_dataset, batch_size=1)
    all_preds, all_refs = [], []
    em_total, f1_total = 0, 0

    for idx, batch in enumerate(tqdm(loader, desc=f"Evaluating {name}")):
        input_ids = batch[f"{name.lower()}_input_ids"].to(device)
        attention_mask = batch[f"{name.lower()}_attention_mask"].to(device)
        prompt = batch["prompt"]
        true_answer = batch["answer"]

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=64)
            prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        generated_answer = prediction.replace(prompt[0], "").strip()
        reference = true_answer[0].strip()

        em = int(exact_match(generated_answer, reference))
        f1 = compute_f1(generated_answer, reference)
        em_total += em
        f1_total += f1

        results.append({
            "prompt": prompt[0],
            "reference": reference,
            f"{name} Prediction": generated_answer,
            "EM": em,
            "F1": f1
        })

        all_preds.append(generated_answer)
        all_refs.append(reference)

    return results, {
        "em": em_total / len(eval_dataset),
        "f1": f1_total / len(eval_dataset)
    }


def print_metrics(name, scores):
    print(f"\n{name} Evaluation:")
    print(f"  Exact Match (EM): {scores['em']:.2f}")
    print(f"  F1 Score: {scores['f1']:.2f}")


def main():
    print(" Loading dataset...")
    dataset = load_adversarialqa_sample(100)

    print(" Loading teacher model...")
    teacher_tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL)
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token
    teacher_model = AutoModelForCausalLM.from_pretrained(TEACHER_MODEL).to(device)

    print(" Loading student model...")
    student_tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)
    student_tokenizer.pad_token = student_tokenizer.eos_token
    student_pad_token_id = student_tokenizer.pad_token_id
    student_model = AutoModelForCausalLM.from_pretrained(STUDENT_MODEL).to(device)

    print(" Preparing dataset...")
    kd_dataset = KnowledgeDistillationDataset(dataset, teacher_tokenizer, student_tokenizer, MAX_LENGTH)
    train_size = int(0.8 * len(kd_dataset))
    train_dataset, eval_dataset = random_split(kd_dataset, [train_size, len(kd_dataset) - train_size])
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    optimizer = AdamW(student_model.parameters(), lr=LEARNING_RATE)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*NUM_EPOCHS)

    for epoch in range(NUM_EPOCHS):
        print(f"\n Epoch {epoch+1}/{NUM_EPOCHS}")
        train_model(teacher_model, student_model, train_loader, optimizer, scheduler, student_pad_token_id)

        out_dir = f"student_checkpoint_epoch_{epoch+1}"
        os.makedirs(out_dir, exist_ok=True)
        student_model.save_pretrained(out_dir)
        student_tokenizer.save_pretrained(out_dir)

    print("\n Evaluating models...")
    student_results, student_scores = evaluate_model(student_model, student_tokenizer, eval_dataset, name="Student")
    teacher_results, teacher_scores = evaluate_model(teacher_model, teacher_tokenizer, eval_dataset, name="Teacher")

    print_metrics("Student", student_scores)
    print_metrics("Teacher", teacher_scores)

    # Combine both predictions in one dataframe
    merged_results = []
    for s, t in zip(student_results, teacher_results):
        merged_results.append({
            "Prompt": s["prompt"],
            "True Answer": s["reference"],
            "Student Prediction": s["Student Prediction"],
            "Teacher Prediction": t["Teacher Prediction"],
            "EM": s["EM"],
            "F1": s["F1"]
        })

    pd.DataFrame(merged_results).to_csv("distillation_eval_results_adversarialqa.csv", index=False)
    print("\n Results saved to distillation_eval_results_adversarialqa.csv")


if __name__ == "__main__":
    main()


Using device: cuda
 Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.35M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/495k [00:00<?, ?B/s]

...


 Epoch 3/3


Training: 100%|██████████| 80/80 [00:46<00:00,  1.71it/s, Loss=49.6, CE=9.4, KD=89.9]



 Evaluating models...


Evaluating Teacher: 100%|██████████| 20/20 [00:59<00:00,  2.96s/it]


Student Evaluation:
  Exact Match (EM): 0.00
  F1 Score: 0.04

Teacher Evaluation:
  Exact Match (EM): 0.00
  F1 Score: 0.06

 Results saved to distillation_eval_results_adversarialqa.csv
